In [3]:
import numpy as np
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('seaborn-deep')
from mpl_toolkits import mplot3d as mpl3d
from scipy import signal, stats, linalg, fft

# import our 'specsens' package with the help of 'sys'
import sys
sys.path.insert(0, '..')
import specsens as ss

In [4]:
_ = ss.estimation_comparison_sim(
    gens=50,  # generations, number of environments
    itrs=20,  # iterations, number of tests in each environment
    f_sample=1e6,  # in Hz
    signal_power=1.0,  # in dB
    noise_power=10.0,  # in dB
    length_sec=0.008192,  # length of each sample in seconds
    #     length_sec=0.001024,  # length of each sample in seconds
    noise_uncert=10.0,  # standard deviation of the noise normal distribution in dB
    num_procs=None,  # number of processes to run in parallel
    seed=None,  # set seed for reproducibility
    window='box',  # window used with fft
    fft_len=8192,  # samples used for fft
    #     fft_len=1024,  # samples used for fft
    num_bands=4,  # total number of bands
    f_center=-1.5e5,  # signal center frequency
    band_noise_est=3,  # band to use for noise estimation
    cov_size=20)

---- Simulation parameters ----
Generations:    50
Iterations:     20
Total iters:    1000
Signal power:   1.00 dB
Sig cent. freq: -150000.0 Hz
Noise power:    10.00 dB
Noise uncert:   10.00 dB
SNR:            -9.00 dB
Signal length:  0.008192 s
Signal samples: 8192
FFT length:     8192
Num. of bands:  4
Band noise est: 3
---- Running simulation ----
Using 8 processes on 8 cores


100%|██████████| 50/50 [05:24<00:00,  6.48s/it] 

---- Simulation stats ----
Err time:             -22.2143 dB
Err band:             -19.3585 dB
Err eigenval average: -21.5806 dB
Err eigenval hist:    -20.4419 dB
Err eigenval kde:     -21.6736 dB
Err eigenval mle:     -21.5517 dB
